# Loading data from Digital Earth Africa <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">


In [ ]:
# pip install dist/deafrica_tools-0.1.0.tar.gz

In [ ]:
import datacube
from odc.ui import with_ui_cbk
import datacube
import matplotlib.pyplot as plt
import datacube.utils.rio
from datacube.utils import masking
import odc.algo

from deafrica_tools.plotting import rgb
from deafrica_tools.datahandling import load_ard
from deafrica_tools.dask import create_local_dask_cluster

#This will speed up loading data
datacube.utils.rio.set_default_rio_config(aws='auto', cloud_defaults=True)

%load_ext autoreload
%autoreload 2

In [ ]:
create_local_dask_cluster()

### Connect to the datacube
We then need to connect to the datacube database.
We will then be able to use the `dc` datacube object to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [ ]:
dc = datacube.Datacube(app="03_Loading_data")

## query

In [ ]:
lat, lon = 33.4291, -7.6058
buffer = 0.1
measurements=['red','green','blue', 'swir_1', 'surface_temperature', 'atmospheric_transmittance']

# Create a reusable query
query = {
    'x': (lon-buffer, lon+buffer),
    'y': (lat+buffer, lat-buffer),
    'time': ('1984','2013'),
    'resolution': (-30, 30),
    'dask_chunks':{},
    'output_crs':'epsg:6933',
}

## load_ard

In [ ]:
ld = load_ard(dc=dc,
         products=['ls5_c2l2','ls7_c2l2','ls8_c2l2'],
         measurements=measurements,
         min_gooddata=0.1,
         verbose=True,
         **query)

print(ld)

In [ ]:
ld.compute()

In [ ]:
rgb(ld, index=[1,3,5,10,-1])

In [ ]:
ld.swir_1.plot(col='time', col_wrap=5)

## dc.load

In [ ]:
ds = dc.load(product='ls8_c2l2',
#              measurements=measurements,
             **query
            ).compute()

# ds = dc.load(product='ls7_c2l2',
#              measurements=measurements,
#              **query
#             )

# ds = dc.load(product='ls5_c2l2',
# #              measurements=measurements,
#              **query
#             )


In [ ]:
# for band in ds.data_vars:
#     fig,ax=plt.subplots(1,1, figsize=(5,5))
#     ds[band].isel(time=2).plot()
#     ax.set_title(band)

In [ ]:
# ds.QA_PIXEL.flags_definition

In [ ]:
import pprint
pprint.pprint(ds.QA_PIXEL.flags_definition)

In [ ]:
quality_flags_prod = {
                "clear": True,
                "cloud_shadow": "not_high_confidence",
                "nodata": False,
            }

# "cirrus":"not_high_confidence"
pq_mask = masking.make_mask(ds['QA_PIXEL'], **quality_flags_prod)


In [ ]:
ds_masked = odc.algo.keep_good_only(ds, where=pq_mask)#.compute()
ds_masked = odc.algo.to_float(ds_masked, dtype='float32').compute()

In [ ]:
st = ds_masked['ST_B6'] * 0.00341802 + 149.0 - 273.15

In [ ]:
for i in range(len(ds.time)):
    fig,ax=plt.subplots(1,4, figsize=(15,5), sharey=True)
    pq_mask.isel(time=i).plot(ax=ax[1], add_colorbar=False)
    rgb(ds, index=i, ax=ax[0])
    rgb(ds_masked, index=i, ax=ax[2])
    st.isel(time=i).plot(ax=ax[3])   
       